In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format
import os 

spark = SparkSession.builder.appName("EDA with PySpark").getOrCreate()

In [11]:
os.getcwd()

'/home/victor-gabriel/Documents/repos/picpay-ml-eng/notebook'

In [12]:
df = spark.read.csv("airports-database.csv", header=True, inferSchema=True)

In [20]:
df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [21]:
#get size of the data
print((df.count(), len(df.columns)))

(336776, 21)


In [25]:
#get flight unique values 
df.select("flight").distinct().count()

3844

In [26]:
df[df["flight"] == 1959].show(5)

+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+----+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| 110|2013|    1|  1|   804.0|           810|     -6.0|  1103.0|          1116|    -13.0|     DL|  1959| N947DL|   JFK| MCO|   147.0|     944|   8|    10|2013-01-01 08:00:00|Delta Air Lines Inc.|
| 991|2013|    1|  2|   800.0|           810|    -10.0|  1102.0|          1116|    -14.0|     DL|  1959| N995DL|   JFK| MCO|   143.0|     944|   8|    10|2013-01-02 08:00:00|Delta Air Lines Inc.|
|1935|2013|    1|  3

In [28]:
df.groupBy("flight").count().show(5)

+------+-----+
|flight|count|
+------+-----+
|   496|   10|
|  1959|  330|
|  4935|   32|
|   471|  129|
|  1580|   92|
+------+-----+
only showing top 5 rows



Temos 336776 registros de voos. <br/>
Temos 3844 registros único de voos, ou seja, 3844 voos que foram feitos mais de uma vez e com diferentes aeronaves, atrasos, número da cauda, etc. <br/>

In [31]:
#filter df dep_time = null and arr_time = 
df.filter(df["dep_time"].isNull() & df["arr_time"].isNull()).count() 

8255

8255 registros de voos foram cancelados. <br/>

In [37]:
df.select("dep_delay").describe().show()

+-------+------------------+
|summary|         dep_delay|
+-------+------------------+
|  count|            328521|
|   mean|12.639070257304708|
| stddev| 40.21006089212997|
|    min|             -43.0|
|    max|            1301.0|
+-------+------------------+



In [41]:
not_cancelled_flights = df.filter(df["dep_time"].isNotNull() | df["arr_time"].isNotNull())

In [48]:
not_cancelled_flights.groupBy("dest").count().orderBy("count", ascending=False).show(5)

+----+-----+
|dest|count|
+----+-----+
| ATL|16898|
| ORD|16642|
| LAX|16076|
| BOS|15049|
| MCO|13982|
+----+-----+
only showing top 5 rows



In [49]:
df.groupBy(["origin", "dest"]).count().orderBy("count", ascending=False).show(5)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   JFK| LAX|11262|
|   LGA| ATL|10263|
|   LGA| ORD| 8857|
|   JFK| SFO| 8204|
|   LGA| CLT| 6168|
+------+----+-----+
only showing top 5 rows



Maior tempo médio de atraso

In [52]:
df.groupBy("dest") \
    .mean("dep_delay") \
    .withColumnRenamed("avg(dep_delay)", "avg_dep_delay") \
    .orderBy("avg_dep_delay", ascending=False) \
    .show(5)

+----+------------------+
|dest|     avg_dep_delay|
+----+------------------+
| CAE|35.570093457943926|
| TUL| 34.90635451505017|
| OKC|30.568807339449542|
| BHM| 29.69485294117647|
| TYS|28.493955094991364|
+----+------------------+
only showing top 5 rows



Dia da semana com maior número de voos

In [59]:
df.select(['time_hour']).show()
df_with_weekday = df.withColumn(
    "day_of_week",
    date_format(col("time_hour"), "EEEE")
)

+-------------------+
|          time_hour|
+-------------------+
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 05:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
|2013-01-01 06:00:00|
+-------------------+
only showing top 20 rows



In [62]:
df_with_weekday.select(["time_hour", "day_of_week"]).show(5)

+-------------------+-----------+
|          time_hour|day_of_week|
+-------------------+-----------+
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 05:00:00|    Tuesday|
|2013-01-01 06:00:00|    Tuesday|
+-------------------+-----------+
only showing top 5 rows



In [65]:
df_with_weekday.groupBy("day_of_week").count().orderBy("count", ascending=False).show()

+-----------+-----+
|day_of_week|count|
+-----------+-----+
|     Monday|50690|
|    Tuesday|50422|
|     Friday|50308|
|   Thursday|50219|
|  Wednesday|50060|
|     Sunday|46357|
|   Saturday|38720|
+-----------+-----+

